# Preparación del entorno: instalaciones y librerías

In [1]:
#Instalaciones
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
#Librerías
import os
import pandas as pd
import glob
import shutil
from google.colab import drive
import torch.nn.functional as F
from torch.optim import Adam

#Semilla
SEED = 42

#Librerías donde fijar semilla para garantizar reproducibilidad
import random
random.seed(SEED)
import numpy as np
np.random.seed(SEED)
import torch
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
import transformers
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding, EarlyStoppingCallback, set_seed
set_seed(SEED)
from datasets import load_dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, f1_score
from sklearn.model_selection import ParameterGrid

# Dataset

In [3]:
#Carga del dataset de España
dataset_es = load_dataset("alttal/balanced_SA_opiniones_indumentaria_ES_Espanya_splits")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3006 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/625 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/313 [00:00<?, ? examples/s]

In [4]:
dataset_es

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 3006
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 625
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 313
    })
})

# Métricas de evaluación

In [5]:
#Definimos las métricas para evaluar el modelo
#Este conjunto de métricas depende de la tarea.
#Para clasificación de textos se suelen utilizar accuracy, precision, recall y F1-score.

def compute_metrics(pred):
  y_true = pred.label_ids              # son las etiquetas reales
  y_pred = pred.predictions.argmax(-1) # son las predicciones
  acc = accuracy_score(y_true, y_pred)
  precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')

  #métricas por clase
  precision_per_class, recall_per_class, f1_per_class, _ = precision_recall_fscore_support(y_true, y_pred, average=None)

  #conversiones necesarias para evitar errores de registro en Tensorboard
  precision_per_class = list(precision_per_class)
  recall_per_class = list(recall_per_class)
  f1_per_class = list(f1_per_class)

  return {
      'accuracy': acc,
      'f1': f1,
      'precision': precision,
      'recall': recall,
      'precision_per_class': precision_per_class,
      'recall_per_class': recall_per_class,
      'f1_per_class': f1_per_class
  }

# Documentación del progreso

In [6]:
#Montar unidad de disco
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
#Ruta raíz para guardar cada experimento
path = '/content/drive/My Drive/TFM/Experimentos balanced'

#Cambiar directorio actual de trabajo
os.chdir(path)

#Verificar directorio actual
print("Directorio actual:", os.getcwd())

Directorio actual: /content/drive/My Drive/TFM/Experimentos balanced


In [8]:
#Definición de una función para eliminar logs antiguos (para evitar falta de almacenamiento)
def clean_old_logs(logging_dir, keep_last_n=1):
    log_files = sorted(glob.glob(os.path.join(logging_dir, '*')), key=os.path.getmtime)
    for log_file in log_files[:-keep_last_n]:
        os.remove(log_file)

In [9]:
#Ruta al directorio de logs
log_dir = path

#Eliminar archivos antiguos de TensorBoard (para evitar acumulación de demasiados logs)
for file in os.listdir(log_dir):
    if file.startswith('events.out.tfevents'):
        os.remove(os.path.join(log_dir, file))

# Modelo 2

##Carga del modelo 2 y adaptación de los datos

In [10]:
#Carga del modelo pre-entrenado
model2 = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest", num_labels=3)

config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
#A modo de información, vemos la configuración del modelo
model2.config

RobertaConfig {
  "_name_or_path": "cardiffnlp/twitter-roberta-base-sentiment-latest",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "negative",
    "1": "neutral",
    "2": "positive"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "negative": 0,
    "neutral": 1,
    "positive": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.42.4",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

In [12]:
#¿Cómo son las etiquetas del modelo? Importante porque deben ser iguales a las del dataset
model2.config.id2label

{0: 'negative', 1: 'neutral', 2: 'positive'}

##Tokenización del dataset

In [13]:
#Carga del tokenizador específico
tokenizer2 = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [25]:
#Se define tokenizador del primer modelo
#Importante: a pesar de que el modelo parece aceptar 514 (ver el parámetro "max_position_embeddings" en la parte de configuración)
#La configuración interna del modelo sólo permite 512. De hecho, si se usa padding = 'longest', al momento de entrenar el modelo, el código no funcionará
def tokenize_function2(example):
    return tokenizer2(example["text"], max_length = 512, padding='max_length', truncation=True)

In [26]:
#Tokenización del dataset
encoded_dataset_es_1 = dataset_es.map(tokenize_function2, batched=True)
#Luego de tokenizar se puede eliminar la columna "text" porque el modelo no la necesita
encoded_dataset_es_1 = encoded_dataset_es_1.remove_columns(["text"])
encoded_dataset_es_1

Map:   0%|          | 0/3006 [00:00<?, ? examples/s]

Map:   0%|          | 0/625 [00:00<?, ? examples/s]

Map:   0%|          | 0/313 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 3006
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 625
    })
    validation: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 313
    })
})

In [28]:
#Ejemplo de la tokenización en cada split
print(encoded_dataset_es_1['train'][0])
print(encoded_dataset_es_1['validation'][0])
print(encoded_dataset_es_1['test'][0])

{'label': 0, 'input_ids': [0, 9839, 162, 16954, 118, 542, 102, 843, 1192, 2714, 11163, 6764, 102, 162, 2677, 196, 260, 30249, 366, 117, 2677, 196, 260, 10, 8987, 6510, 2955, 897, 18105, 8647, 263, 3729, 5502, 13531, 3304, 260, 3304, 853, 1176, 19928, 366, 117, 3774, 13103, 2802, 493, 43045, 271, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

## Experimento 2.1.

In [29]:
#Documentación
#Definición de directorios para el primer experimento
output_dir = "./outputs/model2_lit"
shutil.make_archive("./outputs/model2_lit", 'zip', output_dir)
logging_dir = "./logs/model2_lit"
shutil.make_archive("./logs/model2_lit", 'zip', output_dir)

#Si directorios no existen, crearlos
os.makedirs(output_dir, exist_ok=True)
os.makedirs(logging_dir, exist_ok=True)

Entrenamiento con hiperparámetros recomendados

In [30]:
#Definición de EarlyStopping
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3,    # Número de épocas para esperar sin mejora
    early_stopping_threshold=0.0  # Umbral para la mejora mínima
)

In [31]:
#Definición de hiperparámetros
training_args_model2_lit = TrainingArguments(
    output_dir=output_dir,           # Directorio de salida
    logging_dir=logging_dir,         # Directorio para los logs
    eval_strategy="steps",           # Estrategia de evaluación
    save_strategy="steps",           # Estrategia de guardado (debe coincidir con la de evaluación)
    save_steps=1000,                 # Número de pasos entre cada guardado
    logging_strategy="steps",        # Estrategia de registro
    logging_steps=1000,              # Número de pasos entre cada registro
    num_train_epochs=3,              # Número de épocas de entrenamiento
    per_device_train_batch_size=16,  # Tamaño del batch de entrenamiento (podría ser también 32)
    per_device_eval_batch_size=16,   # Tamaño del batch de evaluación (podría ser también 32)
    learning_rate=3e-5,              # Tasa de aprendizaje
    weight_decay=0.01,               # Decadencia de peso (podría ser también 0.1)
    warmup_ratio=0.1,                # Proporción de épocas de calentamiento
    save_total_limit=1,              # Número máximo de checkpoints a guardar (para evitar problemas de almacenamiento)
    seed=SEED,                         # Semilla para garantizar reproducibilidad
    fp16=True,                       # Uso de fp16 para acelerar el entrenamiento
    load_best_model_at_end=True,     # Cargar el mejor modelo al final del entrenamiento
    metric_for_best_model="f1",      # Métrica objetivo, a optimizar para encontrar el mejor modelo
)

In [32]:
#Definición de optimizador
optimizer = Adam(model2.parameters(), lr=3e-5, weight_decay=0.01)

In [33]:
#Definición de un objeto de la clase Trainer para el primer experimento con el modelo 1
trainer_model2_lit = Trainer(
    model = model2,
    args = training_args_model2_lit,
    train_dataset = encoded_dataset_es_1['train'],
    eval_dataset = encoded_dataset_es_1['validation'],
    compute_metrics=compute_metrics,
    tokenizer = tokenizer2,
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer2),
    optimizers = (optimizer, None),
    callbacks = [early_stopping])

Entrenamiento

In [34]:
trainer_model2_lit.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=564, training_loss=0.7643168023292054, metrics={'train_runtime': 73.0698, 'train_samples_per_second': 123.416, 'train_steps_per_second': 7.719, 'total_flos': 2372756801071104.0, 'train_loss': 0.7643168023292054, 'epoch': 3.0})

Validación

In [35]:
trainer_model2_lit.evaluate()

Trainer is attempting to log a value of "[0.9042553191489362, 0.38202247191011235, 0.8923076923076924]" of type <class 'list'> for key "eval/precision_per_class" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.68, 0.7555555555555555, 0.8111888111888111]" of type <class 'list'> for key "eval/recall_per_class" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.776255707762557, 0.5074626865671642, 0.8498168498168499]" of type <class 'list'> for key "eval/f1_per_class" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.6361433267593384,
 'eval_accuracy': 0.7507987220447284,
 'eval_f1': 0.7111784147155237,
 'eval_precision': 0.7261951611222469,
 'eval_recall': 0.748914788914789,
 'eval_precision_per_class': [0.9042553191489362,
  0.38202247191011235,
  0.8923076923076924],
 'eval_recall_per_class': [0.68, 0.7555555555555555, 0.8111888111888111],
 'eval_f1_per_class': [0.776255707762557,
  0.5074626865671642,
  0.8498168498168499],
 'eval_runtime': 0.8711,
 'eval_samples_per_second': 359.308,
 'eval_steps_per_second': 22.959,
 'epoch': 3.0}

Reporte de clasificación (sobre conjunto de validación)

In [36]:
#Obtención de predicciones del conjunto de validación
predictions = trainer_model2_lit.predict(encoded_dataset_es_1['validation'])
y_true = predictions.label_ids
y_pred = predictions.predictions.argmax(-1)

print("Predicciones:")

#Generación del reporte (diccionario)
reporte_dict = classification_report(y_true, y_pred, output_dict=True)

#Reporte completo (texto)
print("Reporte de Clasificación:")
print(classification_report(y_true, y_pred))

#Preparación de datos para el DataFrame
metrics_per_class = []
for label, metrics in reporte_dict.items():
    if label.isdigit():
        metrics_class = {
            "Clase": label,
            "Precisión": f"{metrics['precision']:.2f}",
            "Recall": f"{metrics['recall']:.2f}",
            "F1-score": f"{metrics['f1-score']:.2f}"
        }
        metrics_per_class.append(metrics_class)

#Creación de DataFrame con métricas por clase
df_metrics_per_class = pd.DataFrame(metrics_per_class)

#Tabla de métricas por clase
print("\nMétricas por Clase:")
print(df_metrics_per_class.to_string(index=False))

Predicciones:
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.90      0.68      0.78       125
           1       0.38      0.76      0.51        45
           2       0.89      0.81      0.85       143

    accuracy                           0.75       313
   macro avg       0.73      0.75      0.71       313
weighted avg       0.82      0.75      0.77       313


Métricas por Clase:
Clase Precisión Recall F1-score
    0      0.90   0.68     0.78
    1      0.38   0.76     0.51
    2      0.89   0.81     0.85


In [37]:
#Limpiar logs antiguos
clean_old_logs(logging_dir, keep_last_n=1)

Test

Reporte de clasificación (sobre conjunto de test)

In [38]:
#Obtención de predicciones del conjunto de test
predictions = trainer_model2_lit.predict(encoded_dataset_es_1['test'])
y_true = predictions.label_ids
y_pred = predictions.predictions.argmax(-1)

print("Predicciones:")

#Generación del reporte (diccionario)
reporte_dict = classification_report(y_true, y_pred, output_dict=True)

#Reporte completo (texto)
print("Reporte de Clasificación:")
print(classification_report(y_true, y_pred))

#Preparación de datos para el DataFrame
metrics_per_class = []
for label, metrics in reporte_dict.items():
    if label.isdigit():
        metrics_class = {
            "Clase": label,
            "Precisión": f"{metrics['precision']:.2f}",
            "Recall": f"{metrics['recall']:.2f}",
            "F1-score": f"{metrics['f1-score']:.2f}"
        }
        metrics_per_class.append(metrics_class)

#Creación de DataFrame con métricas por clase
df_metrics_per_class = pd.DataFrame(metrics_per_class)

#Tabla de métricas por clase
print("\nMétricas por Clase:")
print(df_metrics_per_class.to_string(index=False))

Predicciones:
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.85      0.67      0.75       248
           1       0.32      0.70      0.44        90
           2       0.87      0.72      0.79       287

    accuracy                           0.69       625
   macro avg       0.68      0.69      0.66       625
weighted avg       0.78      0.69      0.72       625


Métricas por Clase:
Clase Precisión Recall F1-score
    0      0.85   0.67     0.75
    1      0.32   0.70     0.44
    2      0.87   0.72     0.79


In [39]:
#Limpiar logs antiguos
clean_old_logs(logging_dir, keep_last_n=1)

## Grid Search

Búsqueda de hiperparámetros optimizados con la técnica Grid Search:

In [40]:
#Definición del espacio de búsqueda
param_grid = {
    'num_train_epochs': [3, 5],
    'per_device_train_batch_size': [16, 32],
    'per_device_eval_batch_size': [16, 32],
    'learning_rate': [1e-5, 2e-5, 3e-5, 5e-5, 1e-4],
    'weight_decay': [0.01, 0.1]
}

grid = ParameterGrid(param_grid)

In [41]:
#Función para calcular el f1-score
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return {
        'f1': f1_score(labels, predictions, average='macro')
    }

In [42]:
#Inicialización de variables para guardar los mejores resultados
best_score = 0
best_params = None

In [45]:
#Documentación
#Definición de directorios para el primer experimento
output_dir = "./outputs/model2_search"
shutil.make_archive("./outputs/model2_search", 'zip', output_dir)
logging_dir = "./logs/model2_search"
shutil.make_archive("./logs/model2_search", 'zip', output_dir)

#Si directorios no existen, crearlos
os.makedirs(output_dir, exist_ok=True)
os.makedirs(logging_dir, exist_ok=True)

In [46]:
#Bucle para probar los diferentes parámetros
for params in grid:
    print(f"Entrenando con parámetros: {params}")

    #Definición de hiperparámetros
    training_args_model2_search = TrainingArguments(
        output_dir=output_dir,                                                  # Directorio de salida
        logging_dir=logging_dir,                                                # Directorio para los logs
        eval_strategy="steps",                                                  # Estrategia de evaluación
        save_strategy="steps",                                                  # Estrategia de guardado (debe coincidir con la de evaluación)
        save_steps=1000,                                                        # Número de pasos entre cada guardado
        logging_strategy="steps",                                               # Estrategia de registro
        logging_steps=1000,                                                     # Número de pasos entre cada registro
        num_train_epochs=params['num_train_epochs'],                            # Número de épocas de entrenamiento
        per_device_train_batch_size=params['per_device_train_batch_size'],      # Tamaño del batch de entrenamiento
        per_device_eval_batch_size=params['per_device_eval_batch_size'],        # Tamaño del batch de evaluación
        learning_rate=params['learning_rate'],                                  # Tasa de aprendizaje
        weight_decay=params['weight_decay'],                                    # Decadencia de peso
        save_total_limit=1,                                                     # Número máximo de checkpoints a guardar (para evitar problemas de almacenamiento)
        seed=SEED,                                                              # Semilla para garantizar reproducibilidad
        fp16=True,                                                              # Uso de fp16 para acelerar el entrenamiento
        load_best_model_at_end=True,                                            # Cargar el mejor modelo al final del entrenamiento
        metric_for_best_model='f1',                                             # Métrica objetivo, a optimizar para encontrar el mejor modelo
    )

    #Definición de un objeto de la clase Trainer para la búsqueda con el modelo 2
    trainer_model2_search = Trainer(
        model=model2,
        args=training_args_model2_search,
        train_dataset=encoded_dataset_es_1['train'],
        eval_dataset=encoded_dataset_es_1['validation'],
        tokenizer=tokenizer2,
        data_collator=DataCollatorWithPadding(tokenizer=tokenizer2),
        compute_metrics=compute_metrics,
        optimizers = (optimizer, None),
        callbacks = [early_stopping]
    )

    #Entrenamiento
    trainer_model2_search.train()

    #Limpiar logs antiguos después del entrenamiento en cada iteración
    clean_old_logs(logging_dir, keep_last_n=1)

    #Validación
    eval_result = trainer_model2_search.evaluate()

    #Guardamos métrica objetivo
    score = eval_result.get('eval_f1', 0)

    #Siempre que la métrica objetivo sea mejor que el score guardado hasta el momento, se lo actualiza
    if score > best_score:
        best_score = score
        best_params = params

#Vemos los resultados del Grid Search
print(f"Mejores parámetros: {best_params}")
print(f"Mejor puntuación F1: {best_score}")

Entrenando con parámetros: {'learning_rate': 1e-05, 'num_train_epochs': 3, 'per_device_eval_batch_size': 16, 'per_device_train_batch_size': 16, 'weight_decay': 0.01}


Step,Training Loss,Validation Loss


Entrenando con parámetros: {'learning_rate': 1e-05, 'num_train_epochs': 3, 'per_device_eval_batch_size': 16, 'per_device_train_batch_size': 16, 'weight_decay': 0.1}


Step,Training Loss,Validation Loss


Entrenando con parámetros: {'learning_rate': 1e-05, 'num_train_epochs': 3, 'per_device_eval_batch_size': 16, 'per_device_train_batch_size': 32, 'weight_decay': 0.01}


...

Entrenando con parámetros: {'learning_rate': 0.0001, 'num_train_epochs': 5, 'per_device_eval_batch_size': 32, 'per_device_train_batch_size': 32, 'weight_decay': 0.01}


Step,Training Loss,Validation Loss


Entrenando con parámetros: {'learning_rate': 0.0001, 'num_train_epochs': 5, 'per_device_eval_batch_size': 32, 'per_device_train_batch_size': 32, 'weight_decay': 0.1}


Step,Training Loss,Validation Loss


Mejores parámetros: {'learning_rate': 1e-05, 'num_train_epochs': 3, 'per_device_eval_batch_size': 16, 'per_device_train_batch_size': 16, 'weight_decay': 0.1}
Mejor puntuación F1: 0.7394500603022571


## Experimento 2.2.

In [47]:
#Documentación
#Definición de directorios para el segundo experimento
output_dir = "./outputs/model2_opt"
shutil.make_archive("./outputs/model2_opt", 'zip', output_dir)
logging_dir = "./logs/model2_opt"
shutil.make_archive("./logs/model2_opt", 'zip', output_dir)

#Si directorios no existen, crearlos
os.makedirs(output_dir, exist_ok=True)
os.makedirs(logging_dir, exist_ok=True)

Entrenamiento con hiperparámetros optimizados

In [48]:
#Definición de EarlyStopping
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3,    # Número de épocas para esperar sin mejora
    early_stopping_threshold=0.0  # Umbral para la mejora mínima
)

In [69]:
#Definición de hiperparámetros (basados en Grid Search)
training_args_model2_opt = TrainingArguments(
    output_dir=output_dir,                                # Directorio de salida
    logging_dir=logging_dir,                              # Directorio para los logs
    eval_strategy="steps",                                # Estrategia de evaluación
    save_strategy="steps",                                # Estrategia de guardado (debe coincidir con la de evaluación)
    save_steps=1000,                                      # Número de pasos entre cada guardado
    logging_strategy="steps",                             # Estrategia de registro
    logging_steps=1000,                                   # Número de pasos entre cada registro
    num_train_epochs=3,                                   # Número de épocas de entrenamiento
    per_device_train_batch_size=16,                       # Tamaño del batch de entrenamiento
    per_device_eval_batch_size=16,                        # Tamaño del batch de evaluación
    learning_rate=1e-05,                                  # Tasa de aprendizaje
    weight_decay=0.1,                                     # Decadencia de peso
    save_total_limit=1,                                   # Número máximo de checkpoints a guardar (para evitar problemas de almacenamiento)
    seed=SEED,                                            # Semilla para garantizar reproducibilidad
    fp16=True,                                            # Uso de fp16 para acelerar el entrenamiento
    load_best_model_at_end=True,                          # Cargar el mejor modelo al final del entrenamiento
    metric_for_best_model="f1",                           # Métrica objetivo, a optimizar para encontrar el mejor modelo
)

In [70]:
#Definición de optimizador
optimizer = Adam(model2.parameters(), lr=params['learning_rate'], weight_decay=params['weight_decay'])

In [71]:
#Definición de un objeto de la clase Trainer para el segundo experimento con el modelo 2
trainer_model2_opt = Trainer(
    model = model2,
    args = training_args_model2_opt,
    train_dataset = encoded_dataset_es_1['train'],
    eval_dataset = encoded_dataset_es_1['validation'],
    compute_metrics=compute_metrics,
    tokenizer = tokenizer2,
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer2),
    optimizers = (optimizer, None),
    callbacks = [early_stopping])

Entrenamiento

In [72]:
trainer_model2_opt.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=564, training_loss=1.0669673622077238, metrics={'train_runtime': 75.0106, 'train_samples_per_second': 120.223, 'train_steps_per_second': 7.519, 'total_flos': 2372756801071104.0, 'train_loss': 1.0669673622077238, 'epoch': 3.0})

Validación

In [73]:
trainer_model2_opt.evaluate()

{'eval_loss': 1.082482099533081,
 'eval_f1': 0.3329083665338645,
 'eval_runtime': 0.8807,
 'eval_samples_per_second': 355.393,
 'eval_steps_per_second': 22.709,
 'epoch': 3.0}

Reporte de clasificación (sobre conjunto de validación)

In [74]:
#Obtención de predicciones del conjunto de validación
predictions = trainer_model2_opt.predict(encoded_dataset_es_1['validation'])
y_true = predictions.label_ids
y_pred = predictions.predictions.argmax(-1)

print("Predicciones:")

#Generación del reporte (diccionario)
reporte_dict = classification_report(y_true, y_pred, output_dict=True)

#Reporte completo (texto)
print("Reporte de Clasificación:")
print(classification_report(y_true, y_pred))

#Preparación de datos para el DataFrame
metrics_per_class = []
for label, metrics in reporte_dict.items():
    if label.isdigit():
        metrics_class = {
            "Clase": label,
            "Precisión": f"{metrics['precision']:.2f}",
            "Recall": f"{metrics['recall']:.2f}",
            "F1-score": f"{metrics['f1-score']:.2f}"
        }
        metrics_per_class.append(metrics_class)

#Creación de DataFrame con métricas por clase
df_metrics_per_class = pd.DataFrame(metrics_per_class)

#Tabla de métricas por clase
print("\nMétricas por Clase:")
print(df_metrics_per_class.to_string(index=False))

Predicciones:
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       125
           1       0.19      0.89      0.32        45
           2       0.79      0.59      0.68       143

    accuracy                           0.40       313
   macro avg       0.33      0.49      0.33       313
weighted avg       0.39      0.40      0.36       313


Métricas por Clase:
Clase Precisión Recall F1-score
    0      0.00   0.00     0.00
    1      0.19   0.89     0.32
    2      0.79   0.59     0.68


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

In [75]:
#Limpiar logs antiguos
clean_old_logs(logging_dir, keep_last_n=1)

Test

Reporte de clasificación (sobre conjunto de test)

In [76]:
#Obtención de predicciones del conjunto de test
predictions = trainer_model2_opt.predict(encoded_dataset_es_1['test'])
y_true = predictions.label_ids
y_pred = predictions.predictions.argmax(-1)

print("Predicciones:")

#Generación del reporte (diccionario)
reporte_dict = classification_report(y_true, y_pred, output_dict=True)

#Reporte completo (texto)
print("Reporte de Clasificación:")
print(classification_report(y_true, y_pred))

#Preparación de datos para el DataFrame
metrics_per_class = []
for label, metrics in reporte_dict.items():
    if label.isdigit():
        metrics_class = {
            "Clase": label,
            "Precisión": f"{metrics['precision']:.2f}",
            "Recall": f"{metrics['recall']:.2f}",
            "F1-score": f"{metrics['f1-score']:.2f}"
        }
        metrics_per_class.append(metrics_class)

#Creación de DataFrame con métricas por clase
df_metrics_per_class = pd.DataFrame(metrics_per_class)

#Tabla de métricas por clase
print("\nMétricas por Clase:")
print(df_metrics_per_class.to_string(index=False))

Predicciones:
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       248
           1       0.18      0.87      0.30        90
           2       0.79      0.52      0.63       287

    accuracy                           0.36       625
   macro avg       0.32      0.46      0.31       625
weighted avg       0.39      0.36      0.33       625


Métricas por Clase:
Clase Precisión Recall F1-score
    0      0.00   0.00     0.00
    1      0.18   0.87     0.30
    2      0.79   0.52     0.63


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

In [77]:
#Limpiar logs antiguos
clean_old_logs(logging_dir, keep_last_n=1)

## Experimento extra 2.3.

In [78]:
#Documentación
#Definición de directorios para el tercer experimento
output_dir = "./outputs/model2_est"
shutil.make_archive("./outputs/model2_est", 'zip', output_dir)
logging_dir = "./logs/model2_est"
shutil.make_archive("./logs/model2_est", 'zip', output_dir)

#Si directorios no existen, crearlos
os.makedirs(output_dir, exist_ok=True)
os.makedirs(logging_dir, exist_ok=True)

Entrenamiento con hiperparámetros originales

In [79]:
#Definición de hiperparámetros estándar
training_args_model2_estandar = TrainingArguments(
    output_dir=output_dir,
    logging_dir=logging_dir,
    eval_strategy="steps",
    save_strategy="steps",
    save_steps=1000,
    logging_strategy="steps",
    logging_steps=1000,
    seed=SEED,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)

In [80]:
#Definición de un objeto de la clase Trainer para el tercer experimento con el modelo 1
trainer_model2_estandar = Trainer(
    model = model2,
    args = training_args_model2_estandar,
    train_dataset = encoded_dataset_es_1['train'],
    eval_dataset = encoded_dataset_es_1['validation'],
    compute_metrics=compute_metrics,
    tokenizer = tokenizer2,
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer2)
)

Entrenamiento

In [81]:
trainer_model2_estandar.train()

Step,Training Loss,Validation Loss,F1
1000,0.909100,0.790469,0.540261


TrainOutput(global_step=1128, training_loss=0.8998273416613856, metrics={'train_runtime': 92.3542, 'train_samples_per_second': 97.646, 'train_steps_per_second': 12.214, 'total_flos': 2372756801071104.0, 'train_loss': 0.8998273416613856, 'epoch': 3.0})

Validación

In [82]:
trainer_model2_estandar.evaluate()

{'eval_loss': 0.7904690504074097,
 'eval_f1': 0.5402610881260678,
 'eval_runtime': 1.0643,
 'eval_samples_per_second': 294.092,
 'eval_steps_per_second': 37.584,
 'epoch': 3.0}

Reporte de clasificación (sobre conjunto de validación)

In [83]:
#Obtención de predicciones del conjunto de validación
predictions = trainer_model2_estandar.predict(encoded_dataset_es_1['validation'])
y_true = predictions.label_ids
y_pred = predictions.predictions.argmax(-1)

print("Predicciones:")

#Generación del reporte (diccionario)
reporte_dict = classification_report(y_true, y_pred, output_dict=True)

#Reporte completo (texto)
print("Reporte de Clasificación:")
print(classification_report(y_true, y_pred))

#Preparación de datos para el DataFrame
metrics_per_class = []
for label, metrics in reporte_dict.items():
    if label.isdigit():
        metrics_class = {
            "Clase": label,
            "Precisión": f"{metrics['precision']:.2f}",
            "Recall": f"{metrics['recall']:.2f}",
            "F1-score": f"{metrics['f1-score']:.2f}"
        }
        metrics_per_class.append(metrics_class)

#Creación de DataFrame con métricas por clase
df_metrics_per_class = pd.DataFrame(metrics_per_class)

#Tabla de métricas por clase
print("\nMétricas por Clase:")
print(df_metrics_per_class.to_string(index=False))

Predicciones:
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.69      0.79      0.74       125
           1       0.27      0.07      0.11        45
           2       0.74      0.82      0.77       143

    accuracy                           0.70       313
   macro avg       0.57      0.56      0.54       313
weighted avg       0.65      0.70      0.66       313


Métricas por Clase:
Clase Precisión Recall F1-score
    0      0.69   0.79     0.74
    1      0.27   0.07     0.11
    2      0.74   0.82     0.77


In [84]:
#Limpiar logs antiguos
clean_old_logs(logging_dir, keep_last_n=1)

Test

Reporte de clasificación (sobre conjunto de test)

In [85]:
#Obtención de predicciones del conjunto de test
predictions = trainer_model2_estandar.predict(encoded_dataset_es_1['test'])
y_true = predictions.label_ids
y_pred = predictions.predictions.argmax(-1)

print("Predicciones:")

#Generación del reporte (diccionario)
reporte_dict = classification_report(y_true, y_pred, output_dict=True)

#Reporte completo (texto)
print("Reporte de Clasificación:")
print(classification_report(y_true, y_pred))

#Preparación de datos para el DataFrame
metrics_per_class = []
for label, metrics in reporte_dict.items():
    if label.isdigit():
        metrics_class = {
            "Clase": label,
            "Precisión": f"{metrics['precision']:.2f}",
            "Recall": f"{metrics['recall']:.2f}",
            "F1-score": f"{metrics['f1-score']:.2f}"
        }
        metrics_per_class.append(metrics_class)

#Creación de DataFrame con métricas por clase
df_metrics_per_class = pd.DataFrame(metrics_per_class)

#Tabla de métricas por clase
print("\nMétricas por Clase:")
print(df_metrics_per_class.to_string(index=False))

Predicciones:
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.64      0.79      0.71       248
           1       0.24      0.06      0.09        90
           2       0.74      0.76      0.75       287

    accuracy                           0.67       625
   macro avg       0.54      0.54      0.52       625
weighted avg       0.63      0.67      0.64       625


Métricas por Clase:
Clase Precisión Recall F1-score
    0      0.64   0.79     0.71
    1      0.24   0.06     0.09
    2      0.74   0.76     0.75


In [86]:
#Limpiar logs antiguos
clean_old_logs(logging_dir, keep_last_n=1)

## Experimento final: mejor modelo, nuevo dataset

In [88]:
#Elección del mejor modelo
#Modelo final -> Modelo con hiperparámetros recomendados
#Carga del nuevo dataset: comentarios de Argentina
dataset_ar = load_dataset("alttal/SA_opiniones_indumentaria_ES_Argentina_splits")

Generating train split:   0%|          | 0/2187 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/625 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/313 [00:00<?, ? examples/s]

In [89]:
#Unimos los splits: sólo necesitaremos uno para el test
from datasets import concatenate_datasets
dataset_ar = concatenate_datasets([dataset_ar['train'], dataset_ar['test'], dataset_ar['validation']])

#Ejemplo
print(dataset_ar)

Dataset({
    features: ['id', 'label', 'text', '__index_level_0__'],
    num_rows: 3125
})


In [90]:
#Se eliminan columnas innecesarias
dataset_ar = dataset_ar.remove_columns(["id","__index_level_0__"])
dataset_ar

Dataset({
    features: ['label', 'text'],
    num_rows: 3125
})

In [91]:
#Las etiquetas de los datasets son strings. Se las cambia a valores numéricos
def label_to_int(label):
    if label == "POS":
        return 2
    elif label == "NEG":
        return 0
    else:
        return 1

dataset_ar_1 = dataset_ar.map(lambda x: {"label": label_to_int(x["label"])})

Map:   0%|          | 0/3125 [00:00<?, ? examples/s]

In [92]:
#Tokenización del dataset
encoded_dataset_ar_1 = dataset_ar_1.map(tokenize_function2, batched=True)
#Luego de tokenizar se puede eliminar la columna "text" porque el modelo no la necesita
encoded_dataset_ar_1 = encoded_dataset_ar_1.remove_columns(["text"])
encoded_dataset_ar_1

Map:   0%|          | 0/3125 [00:00<?, ? examples/s]

Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 3125
})

Reporte de clasificación sobre el conjunto de datos de Argentina

In [93]:
#Obtención de predicciones del conjunto de test
predictions = trainer_model2_lit.predict(encoded_dataset_es_1['test'])
y_true = predictions.label_ids
y_pred = predictions.predictions.argmax(-1)

print("Predicciones:")

#Generación del reporte (diccionario)
reporte_dict = classification_report(y_true, y_pred, output_dict=True)

#Reporte completo (texto)
print("Reporte de Clasificación:")
print(classification_report(y_true, y_pred))

#Preparación de datos para el DataFrame
metrics_per_class = []
for label, metrics in reporte_dict.items():
    if label.isdigit():
        metrics_class = {
            "Clase": label,
            "Precisión": f"{metrics['precision']:.2f}",
            "Recall": f"{metrics['recall']:.2f}",
            "F1-score": f"{metrics['f1-score']:.2f}"
        }
        metrics_per_class.append(metrics_class)

#Creación de DataFrame con métricas por clase
df_metrics_per_class = pd.DataFrame(metrics_per_class)

#Tabla de métricas por clase
print("\nMétricas por Clase:")
print(df_metrics_per_class.to_string(index=False))

Predicciones:
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.64      0.79      0.71       248
           1       0.24      0.06      0.09        90
           2       0.74      0.76      0.75       287

    accuracy                           0.67       625
   macro avg       0.54      0.54      0.52       625
weighted avg       0.63      0.67      0.64       625


Métricas por Clase:
Clase Precisión Recall F1-score
    0      0.64   0.79     0.71
    1      0.24   0.06     0.09
    2      0.74   0.76     0.75
